## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2022/08/06 09:36:46 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: './artifacts_local/1/9144d0459c3b4717a1db88854a93242a/artifacts'


/Users/convergeml/engineering/mlops-zoomcamp/02-experiment-tracking/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./artifacts_local/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.list_registered_models()

[]

In [8]:
client.list_run_infos(experiment_id='1')

[<RunInfo: artifact_uri='./artifacts_local/1/9144d0459c3b4717a1db88854a93242a/artifacts', end_time=1659775011162, experiment_id='1', lifecycle_stage='active', run_id='9144d0459c3b4717a1db88854a93242a', run_uuid='9144d0459c3b4717a1db88854a93242a', start_time=1659775006643, status='FINISHED', user_id='convergeml'>]

In [12]:
client.list_run_infos(experiment_id='1')[0].run_id

'9144d0459c3b4717a1db88854a93242a'

In [13]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/08/06 09:40:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1659775217184, current_stage='None', description='', last_updated_timestamp=1659775217184, name='iris-classifier', run_id='9144d0459c3b4717a1db88854a93242a', run_link='', source='./artifacts_local/1/9144d0459c3b4717a1db88854a93242a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>